In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
#from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


2022-05-27 04:59:03.715736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-27 04:59:03.715764: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataframe1 = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_IoT_dataset/Train_Test_IoT_Modbus.csv")
#dataframe2 = pd.read_csv("./datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_IoT_dataset/Train_Test_IoT_Weather.csv")
raw_data1 = dataframe1.values
#raw_data2 = dataframe2.values

In [3]:
data = raw_data1[:, 3:6]
labels = raw_data1[:, -2]

#data = raw_data2[:, 3:5]
#labels = raw_data2[:, -2]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.33, random_state=21)

In [4]:
# len(labels[labels>0]) #16106
# len(labels[labels==0]) #35000


Normalize:

In [5]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)


2022-05-27 04:59:05.147672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-27 04:59:05.147704: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-27 04:59:05.147719: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-05-27 04:59:05.147952: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Separate normal and abnormal classes

In [6]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

#normal_train_data = train_data[~train_labels]
#normal_test_data = test_data[~test_labels]

#anomalous_train_data = train_data[train_labels]
#anomalous_test_data = test_data[test_labels]


Build the model - OVERCOMPLETE AE

In [7]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(64, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Softmax()])
      #layers.Dense(3, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()


Dont use mean squared error in classification; cross entropy yes. However, training the AE with just normal data is not a classification task

In [8]:
autoencoder.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=[tf.keras.metrics.BinaryAccuracy()])

### Regular training with all training instances

In [9]:
history = autoencoder.fit(train_data, train_labels, 
          epochs=50, 
          batch_size=8,
          #validation_data=(test_data, test_labels),
          shuffle=True)


Epoch 1/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6731 - binary_accuracy: 0.6801
Epoch 2/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6299 - binary_accuracy: 0.6800
Epoch 3/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6286 - binary_accuracy: 0.6800
Epoch 4/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6278 - binary_accuracy: 0.6800
Epoch 5/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6278 - binary_accuracy: 0.6800
Epoch 6/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6272 - binary_accuracy: 0.6800
Epoch 7/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6264 - binary_accuracy: 0.6800
Epoch 8/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6270 - binary_accuracy: 0.6800
Epoch 9/50
4281/4281 [==============================] - 5s 1ms/step - loss: 0.6267 - binary_accuracy: 0.6800
Epoch 10/50
4281/42

In [10]:
results = autoencoder.evaluate(test_data, test_labels)
print("test loss, test acc:", results)

528/528 [==============================] - 1s 801us/step - loss: 0.6339 - binary_accuracy: 0.6771
test loss, test acc: [0.633924126625061, 0.6771206855773926]
